# XML

XML est un système d'encodage de données largement répandu qui peut être lu par un humain tout comme par un programme. Il est assez verbeux et certainement pas adapté au stockage de très grandes données, cependant on le rencontre assez souvent pour devoir tôt ou tard s'y attaquer. Voici la bibliothèque [`xml` de Python](https://docs.python.org/3/library/xml.html) qui permet de simplifier le processus.

Le but n'est pas d'expliquer XML ici mais de récupérer dans un fichier XML les données qui nous intéresse. Pour cela on va utiliser le fichier XML des prix des carburants à la pompe (cf https://www.prix-carburants.gouv.fr/rubrique/opendata/ ).

In [1]:
!head -20 data/PrixCarburants_quotidien_20180826.xml  # let see the head of the XML file

<?xml version="1.0" encoding="ISO-8859-1" standalone="yes"?>
<pdv_liste>
  <pdv id="1000001" latitude="4620114" longitude="519791" cp="01000" pop="R">
    <adresse>596 AVENUE DE TREVOUX</adresse>
    <ville>SAINT-DENIS-L�S-BOURG</ville>
    <services>
      <service>Station de gonflage</service>
      <service>Vente de gaz domestique (Butane, Propane)</service>
      <service>Automate CB</service>
    </services>
    <prix nom="Gazole" id="1" maj="2018-08-25T09:56:54" valeur="1425"/>
    <prix nom="SP95" id="2" maj="2018-08-25T09:56:55" valeur="1508"/>
    <prix nom="SP98" id="6" maj="2018-08-25T09:56:55" valeur="1549"/>
    <rupture id="5" nom="E10" debut="2015-02-09T16:19:00" fin=""/>
    <rupture id="3" nom="E85" debut="2017-09-16T09:50:23" fin=""/>
    <rupture id="4" nom="GPLc" debut="2017-09-16T09:50:23" fin=""/>
  </pdv>
  <pdv id="1000002" latitude="4621842" longitude="522767" cp="01000" pop="R">
    <adresse>16 Avenue de Marboz</adresse>
    <ville>BOURG-EN-BRESSE</ville>


Disons que l'on désire récupérer l'adresse postale de toutes les stations qui vendent du sans plomb 95, SP95, et le prix. 

On voit que l'information est stockée dans un arbre dont la racine est `pdv_liste`.
Les informations qui nous intéressent sont dans

* chaque élément `pdv` qui est un point de vente,
* les sous-éléments `adresse`, `ville` et `prix`
* les attributs `cp` de l'élément `pdv` et `valeur` de `prix`

On note qu'il y a plusieurs sous-élements `prix` par point de vente et qu'il faut donc choisir le bon.

Le but est de ranger toutes ces informations dans un tableau ayant pour chaque ligne les champs `adresse`, `cp`, `ville`, `prix` (prix étant ici la valeur du prix).

In [2]:
import xml.etree.ElementTree as ET
tree = ET.parse('data/PrixCarburants_quotidien_20180826.xml')

La racine, obtenue avec `getroot`, est le premier élément.

Vérifions que notre racine s'appelle bien <tt>pdv_liste</tt> et qu'il n'y a pas d'attributs pour cet élément :

In [3]:
root = tree.getroot()
print(root.tag, root.attrib)

pdv_liste {}


En plus de récupérer son nom `tag` et ses attributs `attrib`, pour chaque élément il y a trois cas possibles :

1. l'élément a des sous élements
2. l'élément n'a pas de sous-élements, c'est une feuille, il a un contenu (un texte, un nombre)
3. l'élément a des sous-éléments **et** aussi du contenu qui peut être avant ou après les sous élements.

Les opérations possibles sont suivant les cas :

1. on peut 
    * itérer sur l'élément `for x in element:`
    * chercher un sous-élement avec `find` ou des sous-élements avec `findall`
2. on peut récupérer la valeur de l'élement avec `text`.
3. on peut
    * itérer ou chercher un ou des sous-éléments
    * récupérer la valeur avant les sous-élements avec `text`
    * récupérer la valeur après les sous-élements avec `tail`

In [8]:
for element in root:
    print(element.tag)
    print(element.attrib)
    print(element.find('ville').text)
    break # it would be too long
len(root)

pdv
{'id': '1000001', 'latitude': '4620114', 'longitude': '519791', 'cp': '01000', 'pop': 'R'}
SAINT-DENIS-LèS-BOURG


12635

On peut donc écrire notre programme.

In [5]:
result = []

for element in root:
    cp = element.attrib['cp']
    adresse = element.find('adresse').text
    ville = element.find('ville').text
    for p in element.findall('prix'):
        if p.attrib['nom'] == 'SP95':
            prix = int(p.attrib['valeur']) / 1000
            result.append([adresse, cp, ville, prix])

In [6]:
result[0]

['596 AVENUE DE TREVOUX', '01000', 'SAINT-DENIS-LèS-BOURG', 1.508]

In [7]:
len(result)

5216

On voit que seules 5216 stations sur les 12635 vendent du SP95. Cela est du au fait que le SP95-E10 (E10 dans notre fichier) remplace doucement le SP95.

### Plus

Cette présentation n'est qu'une introduction à XML sous Python. Pour plus d'information comme 

* éviter les failles de sécurité en lisant un fichier XML récupéré sur le web,
* écrire un fichier XML,
* utiliser un schéma qui décrit le XML,
* et d'autres points.

regarder [la documentation et d'autres bibliothèques plus riches](https://wiki.python.org/moin/PythonXml).

{{ PreviousNext("11 datetime.ipynb", "../lesson3 Object Python/01 Classes and objects.ipynb")}}